In [2]:
from pororo import Pororo
import numpy as np
from tqdm import tqdm

from datasets import load_dataset, load_from_disk

from transformers import (
    AutoTokenizer,
    AutoConfig,
)

import torch
from transformers import Trainer, TrainingArguments

mu, sigma = 0, 0.1
ner = Pororo(task="ner", lang="ko")

model_name = 'klue/roberta-large'

# set up tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

2023-01-05 08:16:39 | INFO | pororo.models.brainbert.tasks.sequence_tagging | [input] dictionary: 4005 types
2023-01-05 08:16:39 | INFO | pororo.models.brainbert.tasks.sequence_tagging | [label] dictionary: 41 types


In [3]:
import json
from tqdm import tqdm

def load_data():
    dataset_path = "../dataset/wikipedia_documents.json"
    with open(dataset_path, "r") as f:
        wiki = json.load(f)

    wiki_texts_prev = list(dict.fromkeys([v["text"] for v in wiki.values()]))
    
    wiki_texts = []
    for t in wiki_texts_prev:
        while(len(t) > 500):
            wiki_texts.append(t[:500])
            t = t[384:]  # stride = 500 - 384 
    
    wiki_strings = []
    wiki_labels = []
    wiki_inputs = []
    for text in tqdm(wiki_texts):
        string = ''
        labels = []
        inputs = []
        for pred in ner(text):
            if pred[1] != 'O' and np.random.normal(mu, sigma) > 0:
                mask = tokenizer(pred[0])['input_ids'][1:-1]
                ln = len(mask)
                labels += mask
                inputs += tokenizer('[MASK]' * ln)['input_ids'][1:-1]
                string += '[MASK]' * ln

            else:
                labels += tokenizer(pred[0])['input_ids'][1:-1]
                inputs += tokenizer(pred[0])['input_ids'][1:-1]
                string += pred[0]
        wiki_labels.append(labels)
        wiki_inputs.append(inputs)
        wiki_strings.append(string)
            
    wiki_corpus = [{"document_text": wiki_texts[i], "masked_strings": wiki_strings[i], "inputs":wiki_inputs[i], "labels":wiki_labels[i]} for i in range(len(wiki_texts))]
    return wiki_corpus

def make_data():
    with open("../dataset/wikipedia_documents_ssm_qa.json", "w") as f:
        json.dump(load_data(), f)

In [1]:
make_data()

In [12]:
dataset_path = "../dataset/wikipedia_documents_ssm2.json"
with open(dataset_path, "r") as f:
    wiki = json.load(f)
    print((len(wiki)))
    
for i in range(1):
    print(wiki[i])

63720
{'document_text': '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n* (a) 영속적인 국민\n* (b) 일정한 영토\n* (c) 정부\n* (d) 타국과의 관계 참여 자격.\n특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다.  이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n\n위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다.\n* 일반 국제 승인을 받은 195개 자주 국가.\n** 유엔 가입 국가 193개\n** 성좌의 명칭으로', 'masked_strings': '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 [MASK][MASK]을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다. * ([MASK]) 영속적인 국민 * [MASK][MASK]) 일정한 영토 * (c) [MASK] * (d) 타국과의 관계 참여 자격. 특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 [MASK][MASK][MASK]오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, [MASK]의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 

In [51]:
from datasets import load_dataset

dataset = load_dataset("json", data_files='../dataset/wikipedia_documents_ssm.json')
dataset

2023-01-03 09:53:04 | WARNING | datasets.builder | Using custom data configuration default-e96911525636eee5


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 581.33it/s]


Dataset json downloaded and prepared to /opt/ml/.cache/huggingface/datasets/json/default-e96911525636eee5/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 198.19it/s]


DatasetDict({
    train: Dataset({
        features: ['document_text', 'masked_strings', 'inputs', 'labels'],
        num_rows: 63720
    })
})

In [ ]:
import json
from tqdm import tqdm

def load_data2():
    dataset_path = "./dataset/wikipedia_documents.json"
    with open(dataset_path, "r") as f:
        wiki = json.load(f)

    wiki_texts_prev = list(dict.fromkeys([v["text"] for v in wiki.values()]))
    
    wiki_texts = []
    for t in wiki_texts_prev:
        while(len(t) > 500):
            wiki_texts.append(t[:500])
            t = t[384:]  # stride = 500 - 384 
    
    wiki_strings = []
    wiki_labels = []
    wiki_inputs = []
    for text in tqdm(wiki_texts):
        string = ''
        labels = []
        inputs = []
        for pred in ner(text):
            if pred[1] != 'O' and np.random.normal(mu, sigma) < 0:
                mask = tokenizer(pred[0])['input_ids'][1:-1]
                ln = len(mask)
                labels += mask
                inputs += tokenizer('[MASK]' * ln)['input_ids'][1:-1]
                string += '[MASK]' * ln

            else:
                labels += tokenizer(pred[0])['input_ids'][1:-1]
                inputs += tokenizer(pred[0])['input_ids'][1:-1]
                string += pred[0]
        wiki_labels.append(labels)
        wiki_inputs.append(inputs)
        wiki_strings.append(string)
            
    wiki_corpus = [{"document_text": wiki_texts[i], "masked_strings": wiki_strings[i], "inputs":wiki_inputs[i], "labels":wiki_labels[i]} for i in range(len(wiki_texts))]
    return wiki_corpus

def make_data():
    with open("./dataset/wikipedia_documents_ssm_qa2.json", "w") as f:
        json.dump(load_data(), f)

In [ ]:
make_data()